# Language Detection

In this exercise we try to use the previously learned methods to develop a ngram based Language Detection module.

Language Detection is the task of deciding the language of a given text using only the textual surface. 

The main assumption is that character statistics are very specific for any given language using the same alphabet.
Using language specific corpora we can establish the ngram statistic of this given language by counting all the occuring ngrams.

The assumption for the classification of new texts is that the most probable ngrams of a language are also more likely to appear in any new text.

So the algorithm is:
 - Clean the corpus in a way that it only contains (ascii) letters of the alphabet (no punctuation and digits)
 - Given a corpus of some language, count all the character ngrams and generate a list of the top `k` ngrams. This list should contain unique entries and only character ngrams occurring within tokens (no spaces).
 - Given a new text generate a list of ngrams occurring within the words. (This list does not have to be unique, because the most likely ngrams of a language are also more likely to occurr.)
 - Count how many of these ngrams are among the top k ngrams of all the `known` languages.
 - The output of the algorithm is the language with the most absolute hits.
 
 
As a formula:
Let $ngrams^k_{l_i}$ the list of
the top $k$ ngrams of language $l_i$

Let $ngrams_{input}$ be the list of ngrams of a given input text $$ L(input) = \text{argmax}_i (\{g |g \in l_i ,\text{for g in } ngrams_{input}\}) $$
 
 
Try to distinguish between English and German. 
As English corpus use the complete gutenberg corpus from nltk.
As German corpus use the tagesschau corpus we provided. (Download)


__Deliberations__
 - What are the limits of this approach?
 - Where can you see problems?
 - You could also use Zipf's law for words to classify language by checking new sentences for words in the vocabulary. What would be the difference in approach, advantages and disadvantages?

### Data Loading

In [1]:
import nltk
import pathlib
import matplotlib.pyplot as plt

In [2]:
def load_german_text(path):
    text = ""
    for f in pathlib.Path(path).glob("*.txt"):
        with open(f, "r") as openf:
            text += openf.read()
    return text

In [3]:
german_text = load_german_text("./tagesschau_corpus/")[:1000000]

In [4]:
english_text = nltk.corpus.gutenberg.raw(nltk.corpus.gutenberg.fileids())[:1000000]

In [5]:
english_text[:1000]

"[Emma by Jane Austen 1816]\n\nVOLUME I\n\nCHAPTER I\n\n\nEmma Woodhouse, handsome, clever, and rich, with a comfortable home\nand happy disposition, seemed to unite some of the best blessings\nof existence; and had lived nearly twenty-one years in the world\nwith very little to distress or vex her.\n\nShe was the youngest of the two daughters of a most affectionate,\nindulgent father; and had, in consequence of her sister's marriage,\nbeen mistress of his house from a very early period.  Her mother\nhad died too long ago for her to have more than an indistinct\nremembrance of her caresses; and her place had been supplied\nby an excellent woman as governess, who had fallen little short\nof a mother in affection.\n\nSixteen years had Miss Taylor been in Mr. Woodhouse's family,\nless as a governess than a friend, very fond of both daughters,\nbut particularly of Emma.  Between _them_ it was more the intimacy\nof sisters.  Even before Miss Taylor had ceased to hold the nominal\noffice of 

## Pre-processing

Pre-processing will consist by removing anything not alphabet, space or the german umlauts.

In [7]:
import string
import re
def clean(s: str) -> str:
    valid=string.ascii_letters+string.digits+" äöüß"
    s = s.lower()
    s = "".join([c if c in valid else " " for c in s ])
    s = re.sub("[\s]+", " ", s)
    return s

In [ ]:
print(f"before: {english_text[:100]}")
print(f"after:  {clean(english_text[:100])}")
print(f"before: {german_text[:100]}")
print(f"after:  {clean(german_text[:100])}")

In [9]:
english_text = clean(english_text)
german_text = clean(german_text)

In [10]:
german_text[:100]

'endlich hat ein hilfskonvoi die schwer umkämpfte region um ost ghouta erreicht doch fehlen viele leb'

### N-Grams

In [ ]:
print(list(nltk.ngrams([1, 2, 3, 4], 2)))
print(["-".join(ngram) for ngram in nltk.ngrams("abcdef", 3)])

In [ ]:
print("text:", english_text[:10])
print("first ngram tuple:", next(nltk.ngrams(english_text, 3)))
print("first 10 ngrams:", ["".join(g) for g in nltk.ngrams(english_text, 3)][:10])
print("frequencies for first 200 ngrams:", nltk.FreqDist(["".join(g) for g in nltk.ngrams(english_text, 3)][:200]))

### Classifier

In [23]:
class NGramLanguageClassifier:
    def __init__(self, texts, n=3, topk=1000):
        """ Initialize the classifier """
        self.n = n
        
        # Count ngrams and only keep most common topk ngrams for each language
        counts = {
            k: nltk.FreqDist(self.create_ngrams(v)).most_common(topk) 
            for k, v in texts.items()
        }
        
        # Remove the count, we are only interested in the list of ngrams
        self.dicts = {
            language: [x[0] for x in language_dicts]
            for language, language_dicts in counts.items()
        } 
        
    def create_ngrams(self, texts):
        """Turn a string text into a list of character ngrams"""
        return ["".join(g) for g in nltk.ngrams(texts, self.n)]
    
    def classify(self,text):
        """Classify an incoming text"""
        
        # Generate the ngrams for the input text
        d =  self.create_ngrams(clean(text))
        
        # Allocate an empty dictionary to save the counts of ngram hits for the input text
        counts = {} # Counts dictionary
        
        # Iterate over every language and its ngram lists
        for language, language_dict in self.dicts.items():
            counts[language] = sum([tr in language_dict for tr in d])   

        # Sort list by frequency
        sorted_counts = sorted(list(counts.items()), key=lambda x: -x[1])
        print(sorted_counts)
        # Predict the language with most hits
        return sorted_counts[0][0]

In [ ]:
NGLC = NGramLanguageClassifier({"english":english_text, "german":german_text}, topk=1000)  

### Testing

In [18]:
NGLC.classify("Ich arbeite an der Universität Leipzig.")

[('german', 29), ('english', 21)]


'german'

In [19]:
NGLC.classify("I am working at the University.")

[('english', 23), ('german', 20)]


'english'

In [20]:
NGLC.classify("Ich heiße John")

[('english', 6), ('german', 6)]


'english'

In [21]:
NGLC.classify("dies")

[('german', 2), ('english', 1)]


'german'

### Deliberations

Limits: 
 - Memory problems when all N-grams are stored. This may not be a practical problem, considering the Zipf results.
 - No weighting of the features
 - Dependence on the training corpus
 - possibility to represent non-valid word forms in the language by N-gram combinations

Problems:

- wrong decisions as shown in the examples

Using Words instead of N-grams:
- dictionary of word forms (types)
- more or less stop word check
- always valid word forms
- Problem: often out of vocabulary